In [1]:
from config import MISTRAL_API_KEY
from data_processing.mistral_chat import MistralChat, prompt_template, sys_template

In [2]:
import chess

api_key = MISTRAL_API_KEY
chat = MistralChat(api_key)

# pprint(chat.get_models())
# model_name = "mistral-large-latest"

p = prompt_template(chess.Board())
s = sys_template()
output = chat.chat(prompt=p, system=s)
output

'{\n  "thoughts": [\n    "First, I will examine the available moves for each piece in the opening phase, focusing on developing pieces and controlling the center.",\n    "I have the following options for my first move:",\n    "1. e4: This is a common opening move, controlling the center e5 square and potentially opening lines for my bishops and queen.",\n    "2. d4: This move advances my center control and opens up lines for my queen and bishop on f4.",\n    "3. c4: This move advances my center control and opens up lines for my knight on c3 and potential future bishops on c1 and f3.",\n    "4. Nf3: This move develops my knight and potentially puts pressure on the opponent\'s center e5 square.",\n    "5. g3: This move is less common but can be used to protect my bishop on f1 and potentially open up lines for future g4 pawn advances.",\n    "Based on the information given and the general principles of chess, I will prioritize developing my pieces and controlling the center.",\n    "There

In [20]:
chat.chat(model_name='open-mixtral-8x7b', prompt=p, system=s)

'{\n    "thoughts": "As this is the start of the game, I have all my pieces intact. My immediate goal is to develop my pieces and control the center of the board. I have a few good options: I can move my e-pawn to e4, or my d-pawn to d4. Both moves allow me to control the central squares e4 and d5 (in the case of e4) or d4 and c5 (in the case of d4). Let\'s analyze both moves.\\n\\nMove e4: This move develops my light-squared bishop and queen, while also allowing for potential pawn chains and outposts in the center of the board. However, it does leave the e-pawn potentially exposed to attacks from the dark-squared bishop and queen. But, since I am the first to move, I can create a strong center quickly, forcing Black to react defensively.\\n\\nMove d4: This move also develops my pieces (dark-squared bishop and queen) and allows for central control. It does not expose my pawns quite as much as the e4 move, but it does not create as strong of a central presence initially. However, it doe

In [4]:
import json
json_obj = json.loads(output)
json_obj

{'thoughts': ['First, I will examine the available moves for each piece in the opening phase, focusing on developing pieces and controlling the center.',
  'I have the following options for my first move:',
  '1. e4: This is a common opening move, controlling the center e5 square and potentially opening lines for my bishops and queen.',
  '2. d4: This move advances my center control and opens up lines for my queen and bishop on f4.',
  '3. c4: This move advances my center control and opens up lines for my knight on c3 and potential future bishops on c1 and f3.',
  "4. Nf3: This move develops my knight and potentially puts pressure on the opponent's center e5 square.",
  '5. g3: This move is less common but can be used to protect my bishop on f1 and potentially open up lines for future g4 pawn advances.',
  'Based on the information given and the general principles of chess, I will prioritize developing my pieces and controlling the center.',
  'Therefore, my first move will be e4.',
  

In [5]:
json_obj['thoughts']

['First, I will examine the available moves for each piece in the opening phase, focusing on developing pieces and controlling the center.',
 'I have the following options for my first move:',
 '1. e4: This is a common opening move, controlling the center e5 square and potentially opening lines for my bishops and queen.',
 '2. d4: This move advances my center control and opens up lines for my queen and bishop on f4.',
 '3. c4: This move advances my center control and opens up lines for my knight on c3 and potential future bishops on c1 and f3.',
 "4. Nf3: This move develops my knight and potentially puts pressure on the opponent's center e5 square.",
 '5. g3: This move is less common but can be used to protect my bishop on f1 and potentially open up lines for future g4 pawn advances.',
 'Based on the information given and the general principles of chess, I will prioritize developing my pieces and controlling the center.',
 'Therefore, my first move will be e4.',
 'This move not only op

In [15]:
move_str = json_obj['move']
board = chess.Board()
move = board.parse_san(move_str)
board.push(move)

In [21]:
chat.get_models()

{'object': 'list',
 'data': [{'id': 'open-mistral-7b',
   'object': 'model',
   'created': 1713200199,
   'owned_by': 'mistralai',
   'root': None,
   'parent': None,
   'permission': [{'id': 'modelperm-226be158b1bf47d1921b500cf21e5991',
     'object': 'model_permission',
     'created': 1713200199,
     'allow_create_engine': False,
     'allow_sampling': True,
     'allow_logprobs': False,
     'allow_search_indices': False,
     'allow_view': True,
     'allow_fine_tuning': False,
     'organization': '*',
     'group': None,
     'is_blocking': False}]},
  {'id': 'mistral-tiny-2312',
   'object': 'model',
   'created': 1713200199,
   'owned_by': 'mistralai',
   'root': None,
   'parent': None,
   'permission': [{'id': 'modelperm-642cb100000f492c889b0b2a3773063b',
     'object': 'model_permission',
     'created': 1713200199,
     'allow_create_engine': False,
     'allow_sampling': True,
     'allow_logprobs': False,
     'allow_search_indices': False,
     'allow_view': True,
    

In [7]:
def translate_san_to_sentence(san_move):
    # Castle
    if 'O-O-O' in san_move:
        return f"The king castles queen-side."
    elif 'O-O' in san_move:
        return f"The king castles king-side."
    
    # Pawn moves
    if san_move[0].islower():
        if 'x' in san_move:
            if '=' in san_move:
                file, capture_square, promotion = san_move.split('x')
                promotion_piece = get_promotion_piece(promotion[-1])
                return f"The pawn on {file} captures on {capture_square} and promotes to a {promotion_piece}."
            else:
                file, square = san_move.split('x')
                return f"The pawn on {file} captures the piece on {square}."
        elif '=' in san_move:
            file, promotion = san_move.split('=')
            promotion_piece = get_promotion_piece(promotion[-1])
            return f"The pawn on {file} promotes to a {promotion_piece}."
        else:
            return f"The pawn moves to {san_move}."

    # Piece moves
    piece_map = {'N': 'knight', 'B': 'bishop', 'R': 'rook', 'Q': 'queen', 'K': 'king'}
    piece = piece_map[san_move[0]]

    if len(san_move) > 3 and san_move[1].isalpha() and san_move[2].isdigit():
        # Ambiguity resolution
        disambiguating_file = san_move[1]
        square = san_move[2:]
        if 'x' in square:
            _, capture_square = square.split('x')
            return f"The {piece} on {disambiguating_file}-file captures the piece on {capture_square}."
        else:
            return f"The {piece} on {disambiguating_file}-file moves to {square}."
    else:
        if 'x' in san_move:
            _, square = san_move.split('x')
            return f"The {piece} captures the piece on {square}."
        else:
            square = san_move[1:]
            return f"The {piece} moves to {square}."

def get_promotion_piece(piece_code):
    promotion_map = {'Q': 'queen', 'R': 'rook', 'B': 'bishop', 'N': 'knight'}
    return promotion_map[piece_code]
    

# Example usage
san_moves = [
    'e4', 'd5', 'Nf3', 'Nc6', 'Bb5', 'a6', 'Ba4', 'Nf6', 'O-O', 'Be7', 
    'Re1', 'b5', 'Bb3', 'd4', 'c3', 'O-O', 'cxd4', 'Nxd4', 'Nc3', 'Bb7', 
    'Nxe4', 'Nxe4', 'Rxe4', 'Qd6', 'Re1', 'Rae8', 'Qxd4', 'Qxd4', 'Nxd4', 'Bd6', 
    'g3', 'Rxe1+', 'Nxe1', 'Re8', 'Bg5', 'h6', 'Bh4', 'g5', 'Bg3', 'Ne4', 
    'Bxd6', 'cxd6', 'b3', 'Kg7', 'Bc4', 'f5', 'f3', 'Nc3', 'Kf2', 'Nd1+', 
    'Ke1', 'Ne3', 'Bxe6', 'Rxe6', 'Kd2', 'Nc4+', 'Kc3', 'Ne5', 'f4', 'gxf4', 
    'gxf4', 'Ng4', 'Kd4', 'Re2', 'h3', 'Nh6', 'Ng1', 'Rg2', 'Nh3', 'Rd2+', 
    'Ke5', 'd5', 'b4', 'f4+', 'Ke4', 'Ng4', 'Nf2+', 'Kf5', 'Nh3', 'Kg6', 
    'Rg2', 'Nf2', 'Re2', 'Ng4', 'Rg2', 'Ne3', 'Rc2', 'Nd1', 'Rd2', 'Ne3', 
    'Ke6', 'Kf8', 'Rd3', 'Ke8', 'Nc5', 'Rg3', 'Rd2', 'Rg6+', 'Kf5', 'Nd3', 
    'a4', 'bxa4', 'Rxa4', 'Nc5', 'Ra7', 'Nb3', 'Rf7+', 'Ke8', 'Rb7', 'Nc5', 
    'Rc7', 'Ne6', 'Rc6', 'Kd7', 'Nb6+', 'Kd8', 'Nxa4', 'Nc5', 'Nb6', 'Ne6', 
    'Kb5', 'Kc7', 'Rh6', 'Kb7', 'Nd5', 'Nc5', 'Rh7+', 'Ka6', 'Nc7+', 'Ka5', 
    'Nd5', 'Ne6', 'Rh4', 'f3', 'Kc5', 'd4+', 'Kc4', 'Kb6', 'Rb4+', 'Kc6', 
    'Ne3', 'fxe2', 'Rxd4', 'e1=Q', 'Rd6+', 'Kb5', 'Rd5+', 'Kb4', 'Kd3', 'Qd1+', 
    'Ke4', 'Qc2+', 'Ke5', 'Nd3+', 'Kf6', 'Qd1', 'Rd6+', 'Kf7', 'Rc6', 'Qe2', 
    'Kg6', 'Qf3', 'Rc7+', 'Kh8', 'Nf4+', 'Kg8', 'Ng6', 'Qe8+', 'Kh7', 'Qh5+', 
    'Kg8', 'Qh7#'
]

for move in san_moves:
    sentence = translate_san_to_sentence(move)
    print(sentence)

The pawn moves to e4.
The pawn moves to d5.
The knight moves to f3.
The knight moves to c6.
The bishop moves to b5.
The pawn moves to a6.
The bishop moves to a4.
The knight moves to f6.
The king castles king-side.
The bishop moves to e7.
The rook moves to e1.
The pawn moves to b5.
The bishop moves to b3.
The pawn moves to d4.
The pawn moves to c3.
The king castles king-side.
The pawn on c captures the piece on d4.
The knight captures the piece on d4.
The knight moves to c3.
The bishop moves to b7.
The knight captures the piece on e4.
The knight captures the piece on e4.
The rook captures the piece on e4.
The queen moves to d6.
The rook moves to e1.
The rook moves to ae8.
The queen captures the piece on d4.
The queen captures the piece on d4.
The knight captures the piece on d4.
The bishop moves to d6.
The pawn moves to g3.
The rook captures the piece on e1+.
The knight captures the piece on e1.
The rook moves to e8.
The bishop moves to g5.
The pawn moves to h6.
The bishop moves to h4.


In [8]:
def translate_san_to_sentence(san_move):
    if is_castling_move(san_move):
        return translate_castling_move(san_move)
    if is_pawn_move(san_move):
        return translate_pawn_move(san_move)
    elif is_piece_move(san_move):
        return translate_piece_move(san_move)
    else:
        return "Unknown move format"

def is_pawn_move(san_move):
    return san_move[0].islower()

def translate_pawn_move(san_move):
    if 'x' in san_move:
        if '=' in san_move:
            file, capture_square, promotion = san_move.split('x')
            promotion_piece = get_promotion_piece(promotion[-1])
            return f"{file}-file pawn captures on {capture_square} and promotes to a {promotion_piece}{get_check_or_checkmate(san_move)}."
        else:
            file, square = san_move.split('x')
            return f"{file}-file pawn captures on {square}{get_check_or_checkmate(san_move)}."
    elif '=' in san_move:
        file, promotion = san_move.split('=')
        promotion_piece = get_promotion_piece(promotion[-1])
        return f"pawn to {file[0]}, promoting to a {promotion_piece}{get_check_or_checkmate(san_move)}."
    else:
        return f"pawn to {san_move}{get_check_or_checkmate(san_move)}."

def is_piece_move(san_move):
    return san_move[0].isupper()

def translate_piece_move(san_move):
    piece_map = {'N': 'knight', 'B': 'bishop', 'R': 'rook', 'Q': 'queen', 'K': 'king'}
    piece = piece_map[san_move[0]]

    if len(san_move) > 4 and san_move[1].isalpha() and san_move[2].isdigit():
        disambiguating_file = san_move[1]
        square = san_move[2:]
        if 'x' in square:
            _, capture_square = square.split('x')
            return f"{piece} on {disambiguating_file}-file to capture on {capture_square}{get_check_or_checkmate(san_move)}."
        else:
            return f"{piece} on {disambiguating_file}-file to {square}{get_check_or_checkmate(san_move)}."
    else:
        if 'x' in san_move:
            _, square = san_move.split('x')
            return f"{piece} captures on {square}{get_check_or_checkmate(san_move)}."
        else:
            square = san_move[1:]
            return f"{piece} moves to {square}{get_check_or_checkmate(san_move)}."

def is_castling_move(san_move):
    return san_move in ['O-O', 'O-O-O']

def translate_castling_move(san_move):
    if san_move == 'O-O':
        return "short castling on the king-side."
    elif san_move == 'O-O-O':
        return "long castling on the queen-side."

def get_promotion_piece(piece_code):
    promotion_map = {'Q': 'queen', 'R': 'rook', 'B': 'bishop', 'N': 'knight'}
    return promotion_map[piece_code]

def get_check_or_checkmate(san_move):
    if san_move.endswith('+'):
        return ", giving check"
    elif san_move.endswith('#'):
        return ", resulting in checkmate"
    else:
        return ""

# Example usage
san_moves = [
    'e4', 'd5', 'exd5', 'Qxd5', 'Nc3', 'Qa5', 'Nf3', 'Nf6', 'Bc4', 'c6', 'd3', 'Bg4', 'h3', 'Bh5', 'O-O', 'e6',
    'Be3', 'Bb4', 'Qe2', 'Nbd7', 'Bd2', 'Rd8', 'a3', 'Ba5', 'b4', 'Bb6', 'Na4', 'Qc7', 'Nxb6', 'axb6', 'c4', 'O-O',
    'Rac1', 'h6', 'Be3', 'Rfe8', 'Rfe1', 'Nd5', 'Bd2', 'N7f6', 'a4', 'Qd6', 'Ne5', 'Nh7', 'Bxh6', 'gxh6', 'Qxh5', 'Nxe3',
    'Rxe3', 'Qg6', 'Qxg6+', 'fxg6', 'Rce1', 'Kf7', 'f4', 'Rg8', 'Kf2', 'Rad8', 'Ke2', 'Nf8', 'g3', 'Nd7', 'Kd2', 'Nb8',
    'Nd1', 'Nc6', 'Nc3', 'Rd7', 'Rf1', 'Rh8', 'Nd1', 'Rhd8', 'Ne3', 'Kg7', 'Rff3', 'Rh8', 'Nc2', 'Rhh7', 'Ne1', 'Na5',
    'bxa5', 'bxa5', 'Rb3', 'Rc7', 'Rfb1', 'Kf6', 'Rb6', 'Ra7', 'R1b5', 'Rd2+', 'Ke1', 'Rdd7', 'Kf2', 'Rf7', 'Kg2', 'Rfd7',
    'Kh2', 'Rf7', 'Kg2', 'Ke7', 'Kf2', 'Kd6', 'Ke2', 'c5', 'Kd2', 'Kc6', 'Kc3', 'Kb6', 'Kb3', 'e5', 'fxe5', 'Rxe5',
    'Rxb7+', 'Kxb7', 'Rb6+', 'Kc7', 'Rxa6', 'Kb7', 'Ra5', 'Kb6', 'Rxc5', 'Kb7', 'a5', 'Rfe7', 'Kb4', 'Re4+', 'Kb5', 'R7e5',
    'a6+', 'Ka7', 'Ra5', 'Kb8', 'c5', 'Rc4', 'Rb5', 'Rxc5', 'Rxc5', 'Rd4', 'a7+', 'Kb7', 'Rb5+', 'Kc6', 'Kc4', 'Rd6',
    'Rb6', 'Rxb6', 'cxb6', 'Kxb6', 'Kd5', 'Kxa7', 'Ke6', 'Kb6', 'Kf6', 'a5', 'Kxg6', 'a4', 'Kxh6', 'a3', 'g4', 'a2',
    'g5', 'a1=Q', 'g6', 'Qb2', 'g7', 'Qg2', 'g8=Q', 'Qxg8+', 'Kh7', 'Qg5#'
]

player_colors = ['White', 'Black'] * (len(san_moves) // 2)  # Alternate player colors

move_number = 1
for move, color in zip(san_moves, player_colors):
    turn = f'{move_number}. {move}: {color} plays '
    sentence = translate_san_to_sentence(move)
    print(turn + sentence)
    
    if color == 'Black':
        move_number += 1

1. e4: White plays pawn to e4.
1. d5: Black plays pawn to d5.
2. exd5: White plays e-file pawn captures on d5.
2. Qxd5: Black plays queen captures on d5.
3. Nc3: White plays knight moves to c3.
3. Qa5: Black plays queen moves to a5.
4. Nf3: White plays knight moves to f3.
4. Nf6: Black plays knight moves to f6.
5. Bc4: White plays bishop moves to c4.
5. c6: Black plays pawn to c6.
6. d3: White plays pawn to d3.
6. Bg4: Black plays bishop moves to g4.
7. h3: White plays pawn to h3.
7. Bh5: Black plays bishop moves to h5.
8. O-O: White plays short castling on the king-side.
8. e6: Black plays pawn to e6.
9. Be3: White plays bishop moves to e3.
9. Bb4: Black plays bishop moves to b4.
10. Qe2: White plays queen moves to e2.
10. Nbd7: Black plays knight moves to bd7.
11. Bd2: White plays bishop moves to d2.
11. Rd8: Black plays rook moves to d8.
12. a3: White plays pawn to a3.
12. Ba5: Black plays bishop moves to a5.
13. b4: White plays pawn to b4.
13. Bb6: Black plays bishop moves to b6.
1

In [12]:
from data_processing.prompt_generator import  san_to_str, translate_game

game = chess.pgn.Game()
"""play 10 moves of game"""
translate_game(game)

TypeError: parse_san() missing 1 required positional argument: 'san'

In [28]:
chess.Move.from_uci('Ng8f6')

InvalidMoveError: invalid uci: 'Ng8f6'